<a href="https://colab.research.google.com/github/Zhikharevi/Z.githab.io/blob/master/Python_2_HW_2_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Продвинутый Python, ДЗ-2

Правила игры:

В домашке 8 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

В каждой задаче необходимо реализовать функцию, которая после будет проверяться через github classroom на тестах. Сами тесты лежит в гитхабе, можете локально проверить работу функций перед сдачей

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно использовать pandas

In [ ]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [ ]:
!pip install wget
#перед сдачей это закомментить

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=c5ce47c74f893298d0bc9235a19b0324fe13817c0bc78b25be97eff0b4d1c466
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
# Код, который будет в каждом тесте, названия не менять
import pandas as pd
import numpy as np
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

## Задание 1 (10 баллов)

Определите:

* Число товаров

* Среднюю стоимость товара (стоимость товара = среднее от цен в датасете items)

в разрезе категорий (все категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
category | products | price

value    | value      | value
```

**Обратите внимание:**

Для категории portateis_cozinha_e_preparadores_de_alimentos перевод portable kitchen and food preparers

Для категории pc_gamer перевод PC Gamer

Для них нужно отдельно добавить перевод

In [ ]:
%%write_and_run task_1.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_1(translation, items, products):

    translation_extra = pd.DataFrame([('portateis_cozinha_e_preparadores_de_alimentos', \
                                       'portable kitchen and food preparers'),
                                      ('pc_gamer', 'PC Gamer')],
                                      columns = ['product_category_name', 'product_category_name_english'])

    translation_full = pd.concat((translation, translation_extra), ignore_index = True)

    filtered_products = products[['product_id', 'product_category_name']]
    filtered_items = items[['product_id','price']]

    product_costs = pd.merge(filtered_products, filtered_items, left_on = 'product_id', right_on = 'product_id')
    aggregate_by_product_type = product_costs.groupby(['product_id', 'product_category_name']).agg({'price':'mean'}).reset_index().rename(columns = {'product_id':'products'})
    aggregate_by_category = aggregate_by_product_type.groupby('product_category_name').agg({'products':'count', 'price':'mean'}).reset_index()

    res = pd.merge(translation_full, aggregate_by_category, left_on = 'product_category_name', right_on = 'product_category_name')
    res = res.drop('product_category_name', axis=1)
    res = res.rename(columns = {'product_category_name_english':'category'})

    return res

In [ ]:
#Проверки

res = task_1(translation, items, products)

assert res[res.category == 'portable kitchen and food preparers'].price.values[0] == 186.996
assert len(res) == 73
assert len(res.drop_duplicates()) == 73
assert res[res.category == 'drinks'].products.values[0] == 81
assert res.products.sum() == 32341
assert res.price.sum() == 12459.75144435194
assert res[res.category == 'home_confort'].price.values[0] == 185.56926417326417

## Задание 2 (10 баллов)

Определите для каждого продавца основную категорию их продаж (категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
seller_id | category

value    | value
```




In [ ]:
%%write_and_run task_2.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_2(translation, products, items):

    translation_extra = pd.DataFrame([('portateis_cozinha_e_preparadores_de_alimentos', \
                                   'portable kitchen and food preparers'),
                                  ('pc_gamer', 'PC Gamer')],
                                  columns = ['product_category_name', 'product_category_name_english'])

    translation_full = pd.concat((translation, translation_extra), ignore_index = True)

    filtered_items = items[['seller_id', 'product_id']]
    filtered_products = products[['product_id', 'product_category_name']]

    sellers_products = pd.merge(filtered_items, filtered_products, left_on = 'product_id', \
                                right_on = 'product_id').drop('product_id', axis = 1)

    sellers_products_translated = pd.merge(sellers_products, translation_full, \
                                           left_on = 'product_category_name', right_on = 'product_category_name')
    sellers_products_translated = sellers_products_translated.drop('product_category_name', axis = 1).rename(columns = {'product_category_name_english':'category'})

    sellers_category_counts = sellers_products_translated.groupby(['seller_id', 'category']).agg(cat_count = ("category", "count")).reset_index()

    most_popular_category_ids = sellers_category_counts.groupby(['seller_id']).agg(most_popular_id = ('cat_count','idxmax')).reset_index().most_popular_id.values

    res = sellers_category_counts.drop('cat_count', axis = 1).loc[most_popular_category_ids].reset_index()

    return res

In [ ]:
# Проверки

res = task_2(translation, products, items)

assert res[res.seller_id == 'e3e15e2c0b9700561efac21c6be48066'].category.values[0] == 'housewares'
assert res[res.seller_id == '2f73e04d12cdf0c945ded66bb3fcf6c7'].category.values[0] == 'garden_tools'
assert len(res) == len(res.drop_duplicates())
assert len(res) == 3035
assert len(res[res.category == 'telephony']) == 66
assert list(np.sort(res.groupby("category").agg({"seller_id": "nunique"}).seller_id.values)) == [  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   4,
         4,   4,   4,   5,   5,   5,   5,   5,   5,   6,   6,   6,   7,
         8,  10,  12,  13,  13,  13,  14,  14,  14,  15,  16,  17,  17,
        17,  19,  20,  20,  20,  21,  22,  26,  37,  37,  43,  46,  51,
        54,  59,  66,  66,  78,  87,  87,  99, 101, 116, 125, 156, 216,
       224, 256, 288, 310]

AssertionError: ignored

## Задание 3 (10 баллов)

Выведите долю потраченных денег по каждому штату (потраченные деньги - сумма денег по доставленным заказам, сумма по price и freight_value)

*Примечание:* разбивка по штатам - по штату покупателя, процент - число от 0 до 1


Табличка, которая у вас должна получиться:

```
state | perc

value | value
```

In [ ]:
%%write_and_run task_3.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_3(orders, customers, items):

    vals = np.vstack((items['order_id'].values, items['price'].values + items['freight_value'].values)).T
    order_total_cost = pd.DataFrame(vals, columns = ['order_id','total_cost'])

    delivered_orders = orders[orders['order_status'] == 'delivered'][['customer_id','order_id']]

    orders_with_costs = pd.merge(delivered_orders, order_total_cost,
                             left_on = 'order_id', right_on = 'order_id')

    states_to_costs = pd.merge(orders_with_costs, customers, left_on = 'customer_id', right_on = 'customer_id')[['customer_state', 'total_cost']]

    total_sum = states_to_costs.total_cost.sum()
    res = states_to_costs.groupby('customer_state').agg({'total_cost':'sum'}).reset_index().rename(columns = {'customer_state':'state', 'total_cost':'perc'})
    res['perc'] = res['perc'].apply(lambda x: x/total_sum)
    return res

In [ ]:
# Проверки

res = task_3(orders, customers, items)
assert res.perc.sum() == 1
assert res[res.state == "RS"].perc.values[0] == 0.055868056429816286
assert res.sort_values("perc", ascending=True).iloc[0, 1] == 0.0005862290943146945
assert res.sort_values("perc", ascending=False).iloc[0, 1] == 0.3741756035817322
assert len(res) == 27

AssertionError: ignored

## Задание 4 (10 баллов)

Определите средний чек покупки (добавьте разбивку на стоимость самого заказ и стоимость доставки) и среднее число товаров в заказе

А также определите среднее число покупок на пользователя (обратите внимание на идентификаторы)

In [ ]:
%%write_and_run task_4.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_4(items, orders, customers):
    filtered_items = items[['order_id', 'order_item_id', 'price', 'freight_value']]
    aggregated_items = filtered_items.groupby('order_id').agg(item_count = ('order_item_id', 'count'), total_price = ('price', 'sum'), total_freight = ('freight_value','sum')).reset_index()

    filtered_orders = orders[['customer_id','order_id']]
    filtered_customers = customers[['customer_unique_id', 'customer_id']]

    unique_customers_to_orders = pd.merge(filtered_customers, filtered_orders, left_on = 'customer_id', right_on = 'customer_id').drop('customer_id', axis = 1)

    aggregated_orders = unique_customers_to_orders.groupby('customer_unique_id').agg(total_orders = ('order_id','count')).reset_index()

    res = (aggregated_items.total_price.mean(),
           aggregated_items.total_freight.mean(),
           aggregated_items.item_count.mean(),
           aggregated_orders.total_orders.mean())

    return res

In [ ]:
# Проверка

res = task_4(items, orders, customers)

assert res == (137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0348089410589412)

## Задание 5 (10 баллов)

Посчитайте CSAT (customer satisfaction - средняя оценка ревью) и отобразите средний CSAT по дням в период с апреля 2017 по апрель 2018 года

Все манипуляции с датой необходимо сделать с помощью datetime и dateutil


Табличка, которая у вас должна получиться:

```
date | csat

"YYYY-MM-DD" | value
```

In [ ]:
%%write_and_run task_5.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime

def task_5(reviews):
    filtered_reviews = reviews[['review_creation_date', 'review_score']].rename(columns = {'review_creation_date':'date', 'review_score':'csat'})

    filtered_reviews['date'] = filtered_reviews['date'].apply(lambda x: parse(x).date())
    filtered_reviews['belongs_to_period'] = filtered_reviews['date'].apply(lambda x: (x >= datetime.date(2017, 4, 1)) and (x < datetime.date(2018, 5, 1)))

    filtered_reviews = filtered_reviews.loc[filtered_reviews['belongs_to_period'] == True]

    res = filtered_reviews.groupby('date').agg({'csat':'mean'}).reset_index()
    res['date'] = res['date'].apply(lambda x: str(x))

    return res

In [ ]:
# Проверки
res = task_5(reviews)

assert res.date.min() == '2017-04-01'
assert res.date.max() == '2018-04-30'
assert res.csat.sum() == 1551.8881071384853
assert res[res.date == '2017-07-11'].csat.values[0] == 4.291390728476821
assert res[res.date == '2018-02-09'].csat.values[0] == 3.992156862745098
assert res[res.csat == 3.6814814814814816].date.values[0] == '2018-02-25'

## Задание 6 (10 баллов)

Посмотрите, как быстро отвечают пользователи (сделайте аггреграцию по числу дней ответа) и какая средняя оценка

Все манипуляции со временем нужно делать через datetime и dateutil

Табличка, которая у вас должна получиться:

```
days | csat | orders

value | value | value
```

Результаты должны быть отсориртированы по дня по возрастанию

In [ ]:
%%write_and_run task_6.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime

def task_6(reviews):
    filtered_reviews = reviews[['review_creation_date', 'review_answer_timestamp', 'review_score', 'order_id']].rename(columns = {'review_creation_date':'creation_date', 'review_answer_timestamp':'answer_date','review_score':'csat'})
    filtered_reviews['creation_date'] = filtered_reviews['creation_date'].apply(lambda x: parse(x).date())
    filtered_reviews['answer_date'] = filtered_reviews['answer_date'].apply(lambda x: parse(x).date())

    filtered_reviews['days'] = (filtered_reviews['answer_date'] - filtered_reviews['creation_date']).apply(lambda x: x.days)
    filtered_reviews = filtered_reviews[['days', 'csat', 'order_id']]

    aggregated_data = filtered_reviews.groupby('days').agg(csat = ('csat', 'mean'), orders = ('order_id', 'count')).reset_index()
    res = aggregated_data.sort_values(0, axis = 1)
    return res

In [ ]:
# Проверки
res = task_6(reviews)
assert res.orders.sum() == 99224
assert np.all(res.days.values == np.sort(res.days.values))
assert len(res) == 214
assert res.days.min() == 0
assert res.days.max() == 518
assert res[res.days == 233].csat.values[0] == 3.0
assert res[res.days == 87].orders.values[0] == 4

## Задание 7 (10 баллов)

Выделите все заказы, где не проставлено поле order_delivered_customer_date. Замените его на дату '2999-12-31'

In [ ]:
%%write_and_run task_7.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_7(orders):
    res = orders.copy()
    res['order_delivered_customer_date'].fillna(value = '2999-12-31', inplace = True)

    return res

In [ ]:
#Проверки

res = task_7(orders)
assert len(res[res.order_delivered_customer_date.isna()]) == 0
assert len(res) == 99441
assert len(res[res.order_delivered_customer_date == '2999-12-31']) == 2965

## Задание 8 (30 баллов)

Определите топ-10 продавцов, у которых больше 100 заказов, отсылающие чаще всего свою посылку в другие регионы (считаются только доставленные заказы)

Чаще всего отсылают = самый большой процент отправленных заказов в другой штат

Табличка, которая у вас должна получиться:

```
seller_id | share

value | value
```

In [ ]:
%%write_and_run task_8.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_8(orders, items, sellers, customers):
    delivered_orders = orders[orders['order_status'] == 'delivered']['order_id']
    filtered_items = items[['order_id', 'seller_id']]

    sellers_to_delivered_orders = pd.merge(delivered_orders,
                                           filtered_items,
                                           left_on = 'order_id', right_on = 'order_id')

    aggregated_sellers = sellers_to_delivered_orders.groupby('seller_id').agg(order_count = ('order_id', 'count')).reset_index()
    active_sellers = aggregated_sellers[aggregated_sellers['order_count'] > 100]['seller_id']

    filtered_sellers = sellers[['seller_id', 'seller_state']]
    sellers_to_states = pd.merge(active_sellers, filtered_sellers,
                                 left_on = 'seller_id', right_on = 'seller_id')

    filtered_items = items[['seller_id', 'order_id']]

    sellers_to_orders = pd.merge(sellers_to_states, filtered_items,
                                 left_on = 'seller_id', right_on = 'seller_id')

    filtered_orders = orders[orders['order_status'] == 'delivered'][['order_id', 'customer_id']]

    sellers_to_customers = pd.merge(sellers_to_orders, filtered_orders,
                                    left_on = 'order_id', right_on = 'order_id').drop('order_id', axis = 1)

    filtered_customers = customers[['customer_id', 'customer_state']]

    sellers_to_customer_states = pd.merge(sellers_to_customers, filtered_customers,
                                          left_on = 'customer_id', right_on = 'customer_id').drop('customer_id', axis = 1)

    sellers_to_customer_states['is_other_state'] = sellers_to_customer_states[['seller_state', 'customer_state']].apply(lambda x: 1 if x['seller_state'] != x['customer_state'] else 0, axis = 1)

    sellers_to_customer_states = sellers_to_customer_states.groupby('seller_id').agg(count_total = ('seller_state', 'count'),
                                                                            count_other_state = ('is_other_state', 'sum')).reset_index()

    sellers_to_customer_states['share'] = sellers_to_customer_states['count_other_state']/sellers_to_customer_states['count_total']

    res = sellers_to_customer_states[['seller_id', 'share']].sort_values(by = ['share'], ascending = False)[:10]

    return res

In [ ]:
# Проверки

res = task_8(orders, items, sellers, customers)

assert np.all(res.share.values == np.sort(res.share.values)[::-1])
assert res.share.values[0] == 0.9743589743589743
print(res.share.values[-1])
assert res.share.values[-1] == 0.9356435643564357
assert res.seller_id.values[5] == '1b4c3a6f53068f0b6944d2d005c9fc89'
assert res.seller_id.values[2] == '06a2c3af7b3aee5d69171b0e14f0ee87'
assert res.share.sum() == 9.519118744616716

0.9391304347826087


AssertionError: ignored